In [1]:
import pandas as pd
df = pd.read_excel("/kaggle/input/biased/Main.xlsx")

In [2]:
df.head()

,Batch,Source Language,ID,Type,Text,English MT,Arabic MT,Annotator ID,Bias,Propaganda,Type of Propaganda,Type of Bias,Comments
0,B01,English,1,MAIN,Yemen's Houthis have waded into the Israel-Ham...,Yemen's Houthis have waded into the Israel-Ham...,خاض الحوثيون في اليمن الحرب بين إسرائيل وحماس ...,1.0,Biased against Palestine,Not Propaganda,Propaganda Not to be deleted,ضمني,NaN
1,B01,English,2,MAIN,Isreal - Hamas Conflict | Face to Face,Isreal - Hamas Conflict | Face to Face,إسرائيل - الصراع مع حماس | وجها لوجه,4.0,Unbiased,Not Propaganda,Not Propaganda,NaN,NaN
2,B01,English,4,MAIN,Videos show how armed men from Gaza stormed a ...,Videos show how armed men from Gaza stormed a ...,أظهرت مقاطع فيديو كيف اقتحم مسلحون من غزة مهرج...,3.0,Unbiased,Not Propaganda,Propaganda Not to be deleted,NaN,NaN
3,B01,English,7,MAIN,Protest in Aligarh Muslim University in suppor...,Protest in Aligarh Muslim University in suppor...,وقفة احتجاجية في جامعة عليكرة الإسلامية دعما ل...,5.0,Unbiased,Not Propaganda,Not Propaganda,NaN,NaN
4,B01,English,8,MAIN,IDF releases audio recording about misfired ro...,IDF releases audio recording about misfired ro...,الجيش الإسرائيلي ينشر تسجيلًا صوتيًا حول صاروخ...,2.0,Biased against Palestine,Propaganda,Propaganda Must be deleted,ضمني,NaN


In [3]:
df.columns

Index(['Batch', 'Source Language', 'ID', 'Type', 'Text', 'English MT',
       'Arabic MT', 'Annotator ID', 'Bias', 'Propaganda', 'Type of Propaganda',
       'Type of Bias', 'Comments'],
      dtype='object')

In [4]:
df = df.drop(columns=['Annotator ID', 'Text','Batch', 'ID', 'Comments', 'Type', 'Source Language', 'English MT',"Type of Bias","Propaganda","Type of Propaganda"], errors='ignore')

In [5]:
df.head()

,Arabic MT,Bias
0,خاض الحوثيون في اليمن الحرب بين إسرائيل وحماس ...,Biased against Palestine
1,إسرائيل - الصراع مع حماس | وجها لوجه,Unbiased
2,أظهرت مقاطع فيديو كيف اقتحم مسلحون من غزة مهرج...,Unbiased
3,وقفة احتجاجية في جامعة عليكرة الإسلامية دعما ل...,Unbiased
4,الجيش الإسرائيلي ينشر تسجيلًا صوتيًا حول صاروخ...,Biased against Palestine


In [6]:
total_rows = len(df)
print(f"Total number of rows: {total_rows}")

Total number of rows: 13500


In [7]:
df.isna().sum()

Arabic MT       0
Bias         2700
dtype: int64

In [8]:
import numpy as np

In [9]:
df.replace(['', 'None', 'nan'], np.nan, inplace=True)  
df.dropna(subset=['Bias'], inplace=True)

In [10]:
df.isna().sum()

Arabic MT    0
Bias         0
dtype: int64

In [11]:
total_rows = len(df)
print(f"Total number of rows: {total_rows}")

Total number of rows: 10800


In [12]:
print(df['Bias'].cat.categories if df['Bias'].dtype.name == 'category' else df['Bias'].unique())

['Biased against Palestine' 'Unbiased' 'Unclear' 'Biased against others'
 'Biased against Israel' 'Biased against both Palestine and Israel'
 'Not Applicable']


In [13]:
columns_to_check = ['Bias']

for column in columns_to_check:
    print(f"Class counts for '{column}':")
    print(df[column].value_counts())
    print("\n")

Class counts for 'Bias':
Bias
Unbiased                                    6817
Biased against Palestine                    2900
Unclear                                      432
Biased against Israel                        281
Biased against others                        203
Not Applicable                               120
Biased against both Palestine and Israel      47
Name: count, dtype: int64




In [14]:
!pip install emoji

In [15]:
import re
import emoji

def preprocess_text(text):
    # Ensure text is a string
    text = str(text)
    
    # 1. Remove hashtags
    text = re.sub(r'#\w+', '', text)
    
    # 2. Remove full URLs with protocols (e.g., "http://example.com")
    text = re.sub(r'https?://\S+|www\.\S+|bit\.ly\S*', '', text)
    
    # 3. Remove standalone paths without protocols (e.g., "/content/kan-news/defense/629514/")
    text = re.sub(r'/\S+', '', text)
    
    # 4. Remove emails
    text = re.sub(r'\S+@\S+', '', text)
    
    # 5. Remove emojis
    text = emoji.replace_emoji(text, replace='')
    
    # 6. Remove diacritics
    arabic_diacritics = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(arabic_diacritics, '', text)
    
    # 7. Remove Tatweel (ـ)
    text = re.sub(r'ـ', '', text)
    
    # 8. Normalize Arabic text
    text = re.sub(r'[إأآا]', 'ا', text)  # Unify Alif variants
    text = re.sub(r'ة', 'ه', text)  # Replace Taa Marbuta with Haa
    text = re.sub(r'ى', 'ي', text)  # Replace Alef Maqsura with Ya
    
    # 9. Remove repeated characters (e.g., "ممتتتاز" → "ممتاز")
    text = re.sub(r'(.)\1+', r'\1', text)
    
    # Return cleaned text
    return text
# Apply the preprocessing function
df['Arabic MT'] = df['Arabic MT'].apply(preprocess_text)
df['Arabic MT'][9022]

'اولا علي قناه ABC: جلست المرشحه الرئاسيه الجمهوريه نيكي هيلي مع لينسي ديفيس من قناه ABC News، حيث ناقشت مجموعه واسعه من المواضيع بما في ذلك الرئيس السابق. ترامب والحرب بين اسرائيل وحماس والاجهاض وحياتها قبل ان تظهر امام الجمهور. اقرا المزيد: '

In [16]:
!pip install tensorflow
!pip install sentence-transformers

^C
ERROR: Operation cancelled by user
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 7.2 MB/s eta 0:00:00:00:01


In [17]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import BorderlineSMOTE
import numpy as np

model = SentenceTransformer("intfloat/multilingual-e5-large")

X = df['Arabic MT']
y = df['Bias']

X_embeddings = model.encode(X.tolist(), convert_to_numpy=True)

X_train, X_test, y_train, y_test = train_test_split(X_embeddings, y, test_size=0.2, stratify=y, random_state=42)

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

Batches:   0%|          | 0/338 [00:00<?, ?it/s]

In [18]:
from imblearn.over_sampling import BorderlineSMOTE

borderline_smote = BorderlineSMOTE(random_state=42)
X_train_resampled, y_train_resampled = borderline_smote.fit_resample(X_train, y_train)

print("Class distribution after Borderline-SMOTE:")
print(pd.Series(y_train_resampled).value_counts())

Class distribution after Borderline-SMOTE:
Bias
Unbiased                                    5454
Biased against Palestine                    5454
Unclear                                     5454
Biased against Israel                       5454
Biased against others                       5454
Biased against both Palestine and Israel    5454
Not Applicable                              5454
Name: count, dtype: int64


In [19]:
from sklearn.metrics.pairwise import cosine_similarity

original_embeddings = X_train
synthetic_embeddings = X_train_resampled[len(X_train):]

cos_sim = cosine_similarity(original_embeddings, synthetic_embeddings)
average_similarity = np.mean(np.max(cos_sim, axis=0))
print(f"Average Cosine Similarity of Synthetic Samples: {average_similarity:.4f}")

if average_similarity < 0.7:
    print("Warning: Synthetic samples are significantly different from the original data.")
else:
    print("Synthetic samples appear similar to the original data.")

Average Cosine Similarity of Synthetic Samples: 0.9886
Synthetic samples appear similar to the original data.


In [35]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from torch.utils.data import Dataset, DataLoader
import torch

# Define Dataset Class
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        
        # Prepare input for T5
        input_text = f"classify: {text}"
        
        encoding = self.tokenizer(
            input_text,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )
        target_encoding = self.tokenizer(
            label,
            max_length=10,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )
        
        return {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
            "labels": target_encoding["input_ids"].flatten(),
        }

# Tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Prepare Dataset
max_len = 128  # Adjust based on your data
train_dataset = TextClassificationDataset(
    texts=X_train_resampled,
    labels=y_train_resampled.tolist(),
    tokenizer=tokenizer,
    max_len=max_len,
)
val_dataset = TextClassificationDataset(
    texts=X_test.tolist(),
    labels=y_test.tolist(),
    tokenizer=tokenizer,
    max_len=max_len,
)


In [36]:
from torch.utils.data import DataLoader

batch_size = 16

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)


In [37]:
from transformers import AdamW
from torch.nn import CrossEntropyLoss
from tqdm import tqdm

# Load T5 model
model = T5ForConditionalGeneration.from_pretrained("t5-small")
model = model.to("cuda" if torch.cuda.is_available() else "cpu")

# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training Loop
def train_epoch(model, data_loader, optimizer):
    model.train()
    losses = []
    for batch in tqdm(data_loader, desc="Training"):
        input_ids = batch["input_ids"].to(model.device)
        attention_mask = batch["attention_mask"].to(model.device)
        labels = batch["labels"].to(model.device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
        )
        loss = outputs.loss
        losses.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return sum(losses) / len(losses)

# Evaluation Loop
def eval_model(model, data_loader):
    model.eval()
    losses = []
    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Evaluating"):
            input_ids = batch["input_ids"].to(model.device)
            attention_mask = batch["attention_mask"].to(model.device)
            labels = batch["labels"].to(model.device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels,
            )
            loss = outputs.loss
            losses.append(loss.item())
    return sum(losses) / len(losses)


/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [38]:
epochs = 5

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    train_loss = train_epoch(model, train_loader, optimizer)
    val_loss = eval_model(model, val_loader)
    print(f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")


Epoch 1/5


Evaluating: 100%|██████████| 135/135 [01:00<00:00,  2.23it/s]


Train Loss: 0.4314, Val Loss: 0.2007
Epoch 2/5


Evaluating: 100%|██████████| 135/135 [01:04<00:00,  2.10it/s]


Train Loss: 0.2023, Val Loss: 0.1704
Epoch 3/5


...

Train Loss: 0.1868, Val Loss: 0.1755
Epoch 4/5


Evaluating: 100%|██████████| 135/135 [01:03<00:00,  2.12it/s]


Train Loss: 0.1765, Val Loss: 0.1940
Epoch 5/5


Evaluating: 100%|██████████| 135/135 [01:03<00:00,  2.13it/s]

Train Loss: 0.1686, Val Loss: 0.1615


In [43]:
from sklearn.metrics import accuracy_score

def eval_model_with_accuracy(model, data_loader, tokenizer):
    model.eval()
    total_correct = 0
    total_samples = 0
    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Evaluating"):
            input_ids = batch["input_ids"].to(model.device)
            attention_mask = batch["attention_mask"].to(model.device)
            labels = batch["labels"].to(model.device)

            # Generate predictions
            outputs = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=10,
            )
            
            # Decode predictions and labels
            predictions = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
            true_labels = [tokenizer.decode(label, skip_special_tokens=True) for label in labels]
            
            # Count correct predictions
            for pred, true_label in zip(predictions, true_labels):
                if pred.strip() == true_label.strip():
                    total_correct += 1
            total_samples += len(true_labels)
    
    # Calculate accuracy
    accuracy = total_correct / total_samples
    return accuracy

# Example usage
accuracy = eval_model_with_accuracy(model, val_loader, tokenizer)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")


Evaluating: 100%|██████████| 135/135 [01:09<00:00,  1.95it/s]

Validation Accuracy: 23.01%
